In [2]:
import pandas as pd
import json
import requests 


headers = {
	"x-rapidapi-host": "api.potterdb.com",
	"Content-Type": "application/json"
}

In [3]:
# Extract relevant information and filter for house characters
def extract_house_characters(house):
    
    # Initialize an empty list to store house characters
    house_characters = []
    
    house_url = f'https://api.potterdb.com/v1/characters?filter[house_cont]={house}'
    

    # Fetch data from the API with pagination
    current_page = 1
    while True:
        response = requests.get(house_url, headers=headers, params={"page[number]": current_page})
        if response.status_code == 200:
            data = response.json()
            for character in data['data']:
                attributes = character['attributes']
                if attributes['house'] == house:
                    house_characters.append({
                        'name': attributes.get('name'),
                        'house': attributes.get('house'),
                        'gender': attributes.get('gender'),
                        'blood_status': attributes.get('blood_status')
                    })
            # Check if there is a next page
            if 'links' in data and 'next' in data['links']:
                current_page += 1
            else:
                break
        else:
            print("Could not get response from API")
            break

    # Create a DataFrame from the collected data
    df_house = pd.DataFrame(house_characters)
    return df_house

In [4]:
hufflepuff_characters = extract_house_characters('Hufflepuff')
hufflepuff_characters

,name,house,gender,blood_status
0,Abraham Grimblehawk,Hufflepuff,Male,Half-blood
1,Adelaide Oakes,Hufflepuff,Female,None
2,Adrian Speke,Hufflepuff,Male,None
3,Agatha Thrussington,Hufflepuff,Female,None
4,Aki Sugiyama,Hufflepuff,Female,None
...,...,...,...,...
193,Wayne Bostock,Hufflepuff,Male,None
194,Wayne Hopkins,Hufflepuff,Male,Half-blood
195,Willa Weholt,Hufflepuff,Female,None
196,Zacharias Smith,Hufflepuff,Male,Pure-blood or half-blood


In [5]:
ravenclaw_characters = extract_house_characters('Ravenclaw')
ravenclaw_characters

,name,house,gender,blood_status
0,Alannis,Ravenclaw,Female,None
1,Alannis Shepley,Ravenclaw,Female,None
2,Amanda,Ravenclaw,Female,None
3,Amina Qureshi,Ravenclaw,Female,None
4,Amit Thakkar,Ravenclaw,Male,None
...,...,...,...,...
189,Xenophilius Lovegood,Ravenclaw,Male,Pure-blood or half-blood
190,Yolanda Reedham,Ravenclaw,Female,None
191,Yutaka Amano,Ravenclaw,Male,None
192,Yvonne Bampton,Ravenclaw,Female,None


In [6]:
slytherin_characters = extract_house_characters('Slytherin')
slytherin_characters

,name,house,gender,blood_status
0,Abigail Pugh,Slytherin,Female,None
1,Abraham Ronen,Slytherin,Male,None
2,Abraxas Malfoy,Slytherin,Male,Pure-blood
3,Adelaide Murton,Slytherin,Female,None
4,Adrian Pucey,Slytherin,Male,None
...,...,...,...,...
248,Yatin Bhagat,Slytherin,Male,None
249,Yubert Thorne,Slytherin,Male,Pure-blood
250,Yurika Haneda,Slytherin,Female,None
251,Zoe Accrington,Slytherin,Female,None


In [7]:
gryffindor_characters = extract_house_characters('Gryffindor')
gryffindor_characters

,name,house,gender,blood_status
0,Aamir Loonat,Gryffindor,Male,None
1,Abigail Grey,Gryffindor,Female,Half-blood
2,Abigail Nicola,Gryffindor,Female,None
3,Adam Pickering,Gryffindor,Male,None
4,Aiden O'Connor,Gryffindor,Male,None
...,...,...,...,...
286,Winky Crocket,Gryffindor,None,None
287,Yann Fredericks,Gryffindor,Male,None
288,Yuvraj Suri,Gryffindor,Male,None
289,Zakir Akram,Gryffindor,Male,None


In [ ]:
all_house_characters = pd.concat([gryffindor_characters, slytherin_characters, ravenclaw_characters, hufflepuff_characters], ignore_index=True)

# Mapping dictionary to standardize blood statuses
blood_status_mapping = {
    'Half-blood': 'Half-blood',
    'Half-blood (most likely)': 'Half-blood',
    'Half-blood or Muggle-born': 'Mixed-blood',
    'Half-blood or Muggle-born (most likely)': 'Mixed-blood',
    'Half-breed (half-giant)': 'Half-breed',
    'Half-breed (one-eighth-Veela)': 'Half-breed',
    'Half-breed (part-goblin)': 'Half-breed',
    'Half-breed (quarter-Veela) (possibly)': 'Half-breed',
    'Muggle-born': 'Muggle-born',
    'Muggle-born (possibly)': 'Muggle-born',
    'Muggle-born or half-blood (likely)': 'Mixed-blood',
    'Muggle-born or half-blood (possibly)': 'Mixed-blood',
    'Pure-blood': 'Pure-blood',
    'Pure-blood (most likely)': 'Pure-blood',
    'Pure-blood (self-proclaimed)': 'Pure-blood',
    'Pure-blood or Half-blood': 'Mixed-blood',
    'Pure-blood or Half-blood (most likely)': 'Mixed-blood',
    'Pure-blood or Half-blood (possibly)': 'Mixed-blood',
    'Pure-blood or half-blood': 'Mixed-blood',
    'Pure-blood or half-blood (likely)': 'Mixed-blood',
    'Pure-blood or half-blood (most likely)': 'Mixed-blood',
    'Pure-blood or half-blood (possibly)': 'Mixed-blood',
    'Pure-blood or half-blood (presumably)': 'Mixed-blood',
    'Pure-blood, half-blood or Muggle-born': 'Mixed-blood'
}

# Apply the mapping to the DataFrame
all_house_characters['blood_status'] = all_house_characters['blood_status'].map(blood_status_mapping)

## Extracting Spells

In [24]:
def extract_spells_data():
    spells_data = []
    spells_url = 'https://api.potterdb.com/v1/spells'
    
    # Fetch data from the API with pagination
    current_page = 1
    while True:
        response = requests.get(spells_url, headers=headers, params={"page[number]": current_page})
        if response.status_code == 200:
            data = response.json()
            for spell in data['data']:
                attributes = spell['attributes']
                spells_data.append({
                    'name': attributes.get('name'),
                    'category': attributes.get('category'),
                    'incantation': attributes.get('incantation'),
                    'effect': attributes.get('effect'),
                    'creator': attributes.get('creator')
                })
        # Check if there is a next page
            if 'links' in data and 'next' in data['links']:
                current_page += 1
            else:
                break
        else:
            print("Could not get response from API")
            break
    
    # Convert the list of dictionaries to a DataFrame
    spells_df = pd.DataFrame(spells_data)
    return spells_df

In [25]:
df_spells = extract_spells_data()
df_spells

,name,category,incantation,effect,creator
0,Age Line,Charm,None,Prevented people above or below a certain age ...,None
1,Alarte Ascendare,Charm,Alarte Ascendare(a-LAR-tay a-SEN-der-ay),Rockets target upward,None
2,Albus Dumbledore's forceful spell,Spell,None,Great force,None
3,Amplifying Charm,Charm,Sonorus(soh-NOHR-us),Loudens target,None
4,Anapneo,"Healing spell, Vanishment, Charm",Anapneo(ah-NAP-nee-oh),Cleared target's airway by vanishing blockages,None
...,...,...,...,...,...
310,Washing up spell,Charm,None,Cleans dishes,None
311,Water-Making Spell,"Charm, Conjuration",Aguamenti(AH-gwah-MEN-tee),Conjures water,None
312,White sparks,Charm,None,Jet of white sparks,None
313,Windy Spell,Charm,Ventus(VEN-tuss),Jet of spiralling wind,None


## Extracting Potions

In [26]:
def extract_potions_data():
    potions_data = []
    potions_url = 'https://api.potterdb.com/v1/potions'
    
    # Fetch data from the API with pagination
    current_page = 1
    while True:
        response = requests.get(potions_url, headers=headers, params={"page[number]": current_page})
        if response.status_code == 200:
            data = response.json()
            for spell in data['data']:
                attributes = spell['attributes']
                potions_data.append({
                    'name': attributes.get('name'),
                    'effect': attributes.get('effect'),
                    'creator': attributes.get('creator')
                })
        # Check if there is a next page
            if 'links' in data and 'next' in data['links']:
                current_page += 1
            else:
                break
        else:
            print("Could not get response from API")
            break
    
    # Convert the list of dictionaries to a DataFrame
    potions_df = pd.DataFrame(potions_data)
    return potions_df

In [27]:
df_potions = extract_potions_data()
df_potions

,name,effect,creator
0,Ageing Potion,Aged drinker temporarily,None
1,Alihotsy Draught,Uncontrollable laughter,None
2,Amortentia,Love potion that caused a powerful infatuation...,None
3,Angel's Trumpet Draught,Poisonous,None
4,Antidote,None,None
...,...,...,...
163,Wideye Potion,"Awakened victim, Prevented sleeping",None
164,Wiggenweld Potion,"Healed injuries and replenished stamina, Antid...",None
165,Wit-Sharpening Potion,Improved intellect,None
166,Wolfsbane Potion,Allowed werewolves to keep their minds post-tr...,None
